In [1]:
import pandas as pd
import os

# Directory where your Excel files are stored
folder_path = 'All Data'  # Replace with the path to your "All Data" folder

# List to store each file's DataFrame
dataframes = []

# Iterate over each file in the directory
for file_name in os.listdir(folder_path):
    if file_name.endswith('.xlsx'):  # Check for Excel files
        file_path = os.path.join(folder_path, file_name)
        # Read the first 200 rows (excluding header) of each file
        data = pd.read_excel(file_path, usecols=['Question', 'Answer'], nrows=200)
        dataframes.append(data)

# Combine all DataFrames from the list
combined_data = pd.concat(dataframes, ignore_index=True)

# Save the combined data to a new Excel file
output_file = 'smaller-cyber-data.xlsx'
combined_data.to_excel(output_file, index=False)

print('Data extraction complete. File saved as:', output_file)


ValueError: Usecols do not match columns, columns expected but not found: ['answer', 'question'] (sheet: 0)

#Send To HuggingFace

In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [5]:
import pandas as pd
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split

# Replace this with your actual file path
file_path = 'smaller-cyber-data.xlsx'

# Load your dataset
df = pd.read_excel(file_path)

# Rename columns to lowercase
df = df.rename(columns={'Question': 'question', 'Answer': 'answer'})

# Add empty columns
df['input_ids'] = None
df['attention_mask'] = None
df['labels'] = None

# Split the dataset into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.2)

# Create Hugging Face datasets
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Combine them into a DatasetDict
dataset_dict = DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})

# Save the dataset to a local folder (this is necessary for the upload)
dataset_dict.save_to_disk('huggingface_dataset')

# You will need to log in to Hugging Face using their CLI tool before executing the next steps
# Use the command `huggingface-cli login` in your terminal to log in

# Define your dataset repository name on Hugging Face Hub
repo_name = 'cyber-short'  # replace with your desired repository name

# Push the dataset to the Hugging Face Hub
# Note: Replace 'your-username' with your actual Hugging Face username
dataset_dict.push_to_hub(f'shahrukh95/{repo_name}')


README.md: 100%|██████████| 450/450 [00:00<00:00, 1.09MB/s]


CommitInfo(commit_url='https://huggingface.co/datasets/shahrukh95/cyber-short/commit/79843318d90c3ce46ecdb15bd7c81c139d78e849', commit_message='Upload dataset', commit_description='', oid='79843318d90c3ce46ecdb15bd7c81c139d78e849', pr_url=None, pr_revision=None, pr_num=None)

#Convert dataset to Falcon format

In [3]:
import pandas as pd

# Load the Excel file
file_path = 'smaller-cyber-data.xlsx'  # Replace with your actual file path
df = pd.read_excel(file_path)

# Function to format the row into the desired string
def format_row(question, answer):
    return f'<HUMAN>: {question}\n<ASSISTANT>: {answer}'

# Apply the formatting to each row
formatted_data = df.apply(lambda row: format_row(row['Question'], row['Answer']), axis=1)

# Save to a new CSV file
output_csv_path = 'formatted-data.csv'
formatted_data.to_csv(output_csv_path, index=False, header=False)

print(f'Formatted data saved to {output_csv_path}')


Formatted data saved to formatted-data.csv


In [4]:
# Get jsonl file

In [5]:
import csv
import json

# Define the input CSV file and output JSONL file
input_csv = 'formatted-data.csv'
output_jsonl = 'output.jsonl'

# Function to parse the text and extract question and answer
def parse_qa(text):
    parts = text.split('\n')
    question = parts[0].replace('<HUMAN>: ', '').strip()
    answer = parts[1].replace('<ASSISTANT>: ', '').strip()
    return {"question": question, "answer": answer}

# Read the CSV, parse data, and write to JSONL
with open(input_csv, mode='r', encoding='utf-8') as csv_file, \
     open(output_jsonl, mode='w', encoding='utf-8') as jsonl_file:
    
    # CSV reader
    reader = csv.DictReader(csv_file)

    # Process each row
    for row in reader:
        qa_data = parse_qa(row['text'])
        jsonl_file.write(json.dumps(qa_data) + '\n')

print("Conversion complete. Data written to", output_jsonl)


Conversion complete. Data written to output.jsonl
